# Vector fields and developmental trajectories of tissue development in three-dimensional space

In [1]:
import os, warnings
from pathlib import Path

import numpy as np
import dynamo as dyn
import spateo as st
import scanpy as sc
warnings.filterwarnings('ignore')

## Load the data

In [2]:
sample_id = "E7_8h"
os.chdir(f"/media/pc001/Yao/Projects/Project_spateo/mouse_heart/figure6")
cpo = [(41, 1209, 57), (13, 8, -3), (0, 0, 1)]

out_h5ad_path = f"droso_{sample_id}_midgut_morpho/h5ad"
out_image_path = f"droso_{sample_id}_midgut_morpho/image/midgut_morphofield"
Path(out_h5ad_path).mkdir(parents=True, exist_ok=True)
Path(out_image_path).mkdir(parents=True, exist_ok=True)

In [3]:
germ_adata = st.read_h5ad(f"/media/pc001/Yao/Projects/Project_drosophila/Data_v4/migration-hotspot/drosophila_E7_8h_germ_layer_new/h5ad/{sample_id}_germband_v3.h5ad")
tissue_adata = germ_adata[germ_adata.obs["anno_tissue_new"] == "midgut", :]
tissue_adata.X = tissue_adata.layers["X_counts"].copy()
tissue_adata = tissue_adata[tissue_adata.X.sum(axis=1)!=0, tissue_adata.X.sum(axis=0)!=0]
tissue_adata

In [4]:
tissue_pc, _ = st.tdr.construct_pc(
    adata=tissue_adata.copy(),
    spatial_key="3d_align_spatial",
    groupby="anno_tissue_new",
    key_added="tissue",
    colormap={"amnioserosa": "#bdcf32", "CNS": "#ea5545", "hindgut": "#27aeef", "midgut": "#87bc45", "muscle": "#f46a9b", "salivary gland": "#ef9b20",},
)

## Check mapping result

In [5]:
stage2_germ_adata = st.read_h5ad(f"/media/pc001/Yao/Projects/Project_drosophila/Data_v4/migration-hotspot/drosophila_E8_9h_germ_layer/h5ad/E8_9h_germ_layer_v3.h5ad")
stage2_tissue_adata = stage2_germ_adata[stage2_germ_adata.obs["anno_tissue_new"] == "midgut", :]
stage2_tissue_adata.X = stage2_tissue_adata.layers["X_counts"].copy()
stage2_tissue_pc, _ = st.tdr.construct_pc(
    adata=stage2_tissue_adata.copy(),
    spatial_key="3d_align_spatial",
    groupby="anno_tissue_new",
    key_added="tissue",
    colormap={"amnioserosa": "#bdcf32", "CNS": "#ea5545", "hindgut": "#27aeef", "midgut": "#87bc45", "muscle": "#f46a9b", "salivary gland": "#ef9b20",},
)
models_distance = np.asarray([0, 0, -260])
stage2_tissue_pc.points = stage2_tissue_pc.points + models_distance
stage2_tissue_adata

In [6]:
model_lines, _ = st.tdr.construct_align_lines(
            model1_points=tissue_adata.obsm["3d_align_spatial"].copy(),
            model2_points=tissue_adata.obsm["X_cells_mapping"] + models_distance,
            key_added="mapping",
            label="lines",
            color="gainsboro",
            alpha=1.0,
        )

mapping_cpo = [(18, 1370, -141),(-1.8, 5.8, -156.8), (0, 0, 1)]
st.pl.three_d_plot(
        model=st.tdr.collect_models([model_lines, st.tdr.merge_models([tissue_pc, stage2_tissue_pc])]),
        key=["mapping", "tissue"],
        jupyter="static",
        cpo=mapping_cpo,
        background="white",
        window_size=(2560, 2048),
        opacity=[0.1, 1],
        model_style=["wireframe", "points"],
        model_size=[2, 16],
        filename=os.path.join(out_image_path, f"E7-8h-E8-9h_midgut_mapping_z_model_260.pdf"),
    )

## Calculate the developmental Vectorfield

In [7]:
st.tdr.morphofield_sparsevfc(
    adata=tissue_adata,
    spatial_key="3d_align_spatial",
    V_key="V_cells_mapping",
    key_added="VecFld_morpho",
    NX=np.asarray(tissue_adata.obsm['3d_align_spatial']),
    inplace=True,
)

tissue_pc.point_data["vectors"] = tissue_adata.uns["VecFld_morpho"]["V"]

In [11]:
for i, k in zip([0, 2], ["V_x", "V_z"]):
    vector_arrows1,_ = st.tdr.construct_field(
        model=tissue_pc,
        vf_key="vectors",
        arrows_scale_key="vectors",
        n_sampling=None,
        factor=6000,
        key_added=k,
        label=tissue_pc.point_data["vectors"][:, i].flatten(),
        color= "rainbow",
    )
    st.pl.three_d_plot(
        model=st.tdr.collect_models([tissue_pc, vector_arrows1]),
        key=["tissue", k],
        colormap=["gainsboro", "Spectral"],
        opacity=[0.2, 1],
        model_style=["points", "surface"],
        model_size=[16, 5],
        jupyter="static",
        background="white",
        window_size=(2560, 2048),
        cpo=cpo,
        filename=os.path.join(out_image_path, f"midgut_morphofield_vector_arrows_pc_model_{k}.pdf")
    )

## Predict the cell developmentalal trajectory

In [12]:
st.tdr.morphopath(
    adata=tissue_adata,
    # layer="log1p_X",
    vf_key="VecFld_morpho",
    key_added="fate_morpho",
    t_end=10000,
    interpolation_num=50,
    cores=20
)

In [15]:
trajectory_model, _ = st.tdr.construct_trajectory(
    adata=tissue_adata,
    fate_key="fate_morpho",
    n_sampling=800,
    sampling_method="trn",
    key_added="obs_index",
    label=np.asarray(tissue_adata.obs.index), # stage1_tissue_adata.uns["VecFld_morpho"]["V"][:, 2].flatten(),
)
for i, k in zip([0, 2], ["V_x", "V_z"]):
    tissue_adata.obs[k] = tissue_adata.uns["VecFld_morpho"]["V"][:, i].flatten()
    st.tdr.add_model_labels(
        model=trajectory_model,
        key_added=k,
        labels=np.asarray(tissue_adata[np.asarray(trajectory_model.point_data["obs_index"])].obs[k]),
        colormap="Spectral",
        where="point_data",
        inplace=True,
    )
    st.pl.three_d_plot(
        model=st.tdr.collect_models([tissue_pc, trajectory_model]),
        key=["tissue", k],
        opacity=[0.2, .5],
        model_style=["points", "wireframe"],
        model_size=[16, 5],
        colormap=["gainsboro", "Spectral"],
        jupyter="static",
        background="white",
        window_size=(2560, 2048),
        cpo=cpo,
        filename=os.path.join(out_image_path, f"midgut_morphofield_trajectory_pc_model_{k}.pdf")
    )

## Animating cell fate prediction

In [16]:
cells_models, _ = st.tdr.construct_genesis(
    adata=tissue_adata,
    fate_key="fate_morpho",
    n_steps = 100,
    logspace=True,
    t_end=10000,
    label=[tissue_adata.uns["VecFld_morpho"]["V"][:, 0]] * 100,
    color="Spectral"
)
st.pl.three_d_animate(
    models=cells_models,
    stable_model=trajectory_model,
    key="genesis",
    stable_kwargs=dict(
        key="V_x",
        model_style="wireframe",
        model_size=5,
        opacity=0.5,
        colormap="Spectral",
        show_legend=False,
    ),
    filename=os.path.join(out_image_path, f"midgut_morphofield_trajectory_model.mp4"),
    colormap="Spectral",
    model_style="points",
    model_size=16,
    jupyter="static",
    background="white",
    window_size=(2560, 2048),
    cpo=cpo,
    framerate=10)

In [17]:
cells_linear_models, _ = st.tdr.construct_genesis_X(
    stages_X=[tissue_adata.obsm["3d_align_spatial"], tissue_adata.obsm["X_cells_mapping"]],
    n_spacing=100,
    key_added="genesis",
    color="Spectral", 
    label=[tissue_adata.uns["VecFld_morpho"]["V"][:, 0]] * (100 + 1),
)
st.pl.three_d_animate(
    models=cells_linear_models,
    stable_model=trajectory_model,
    key="genesis",
    stable_kwargs=dict(
        key="V_x",
        model_style="wireframe",
        model_size=5,
        opacity=0.5,
        colormap="Spectral",
        show_legend=False,
    ),
    filename=os.path.join(out_image_path, f"midgut_morphofield_trajectory_model_linear.mp4"),
    model_style="points",
    model_size=16,
    colormap="Spectral",
    jupyter="static",
    background="white",
    window_size=(2560, 2048),
    cpo=cpo,
    framerate=10)

## Morphometric features

In [18]:
st.tdr.morphofield_velocity(adata=tissue_adata, vf_key="VecFld_morpho", key_added="morpho_velocity",)
st.tdr.morphofield_acceleration(adata=tissue_adata, vf_key="VecFld_morpho", key_added="morpho_acceleration",)
st.tdr.morphofield_curvature(adata=tissue_adata, vf_key="VecFld_morpho", key_added="morpho_curvature")
st.tdr.morphofield_curl(adata=tissue_adata, vf_key="VecFld_morpho", key_added="morpho_curl",)
st.tdr.morphofield_torsion(adata=tissue_adata, vf_key="VecFld_morpho", key_added="morpho_torsion",)
st.tdr.morphofield_divergence(adata=tissue_adata, vf_key="VecFld_morpho", key_added="morpho_divergence",)
tissue_adata

In [19]:
morphometrics_keys = ["morpho_acceleration", "morpho_curvature", "morpho_curl", "morpho_torsion", "morpho_divergence"]
for mk in morphometrics_keys:
    vector_arrows,_ = st.tdr.construct_field(
        model=tissue_pc,
        vf_key="vectors",
        arrows_scale_key="vectors",
        n_sampling=None,
        factor=6000,
        key_added=mk,
        label=np.asarray(tissue_adata[np.asarray(tissue_pc.point_data["obs_index"])].obs[mk]),
        color="afmhot_r",
    )
    
    st.tdr.add_model_labels(
        model = tissue_pc,
        key_added = mk,
        labels = np.asarray(tissue_adata[np.asarray(tissue_pc.point_data["obs_index"])].obs[mk]),
        colormap = "afmhot_r",
        where = "point_data",
        inplace = True,
    )
    st.pl.three_d_plot(
        model=st.tdr.collect_models([tissue_pc, vector_arrows]),
        key=[mk, mk],
        model_style=["points", "surface"],
        model_size=[16, 5],
        opacity=[0.2, 1],
        colormap=["afmhot_r", "afmhot_r"],
        show_legend=True,
        jupyter="static",
        background="white",
        cpo=cpo,
        window_size=(2560, 2048),
        text=mk,
        filename=os.path.join(out_image_path, f"midgut_morphometrics_{mk}_arrows.pdf")
    )

In [20]:
morphometrics_keys = ["morpho_acceleration", "morpho_curvature", "morpho_curl", "morpho_torsion", "morpho_divergence"]

for mk in morphometrics_keys:
    st.tdr.add_model_labels(
        model=trajectory_model,
        key_added=mk,
        labels=np.asarray(tissue_adata[np.asarray(trajectory_model.point_data["obs_index"])].obs[mk]),
        colormap="Spectral",
        where="point_data",
        inplace=True,
    )
    st.pl.three_d_plot(
        model=st.tdr.collect_models([tissue_pc, trajectory_model]),
        key=[mk, mk],
        model_style=["points", "wireframe"],
        model_size=[16, 5],
        opacity=[0.2, 0.5],
        colormap=["gainsboro", "afmhot_r"],
        show_legend=True,
        jupyter="static",
        background="white",
        cpo=cpo,
        window_size=(2560, 2048),
        text=mk,
        filename=os.path.join(out_image_path, f"midgut_morphometrics_{mk}_trajectory.pdf")
    )

## Save the anndata objects and models

In [21]:
del tissue_adata.uns["morpho_torsion"], tissue_adata.uns["fate_morpho"], tissue_adata.uns["glm_degs"], tissue_adata.uns["log1p"]
tissue_adata.write_h5ad(os.path.join(out_h5ad_path, f"droso_{sample_id}_midgut_v4.h5ad"), compression="gzip")
st.tdr.save_model(model=tissue_pc, filename=os.path.join(out_h5ad_path, "midgut_morphometrics_pc_model.vtk"))
st.tdr.save_model(model=trajectory_model, filename=os.path.join(out_h5ad_path, "midgut_morphometrics_trajectory_model.vtk"))
tissue_adata